In [1]:
%load_ext autoreload
%autoreload 2
%run ./initializer

Added the path (/Users/madalenabarcelo/Desktop/BA/DMT) to sys.path


In [2]:
import torch
import torch.nn as nn
import optuna
from sklearn.model_selection import train_test_split
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt

from data_loading import DataPreprocessor

/Users/madalenabarcelo/Desktop/BA/DMT/notebook/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data_loader = DataPreprocessor()
train_df, val_df_split, test_df = data_loader.load_and_preprocess_data("1d", 0.25, 1,0)

Removed 19 outliers from 968 observations. Percentage: 1.96%


In [24]:


# Drop non-numeric / irrelevant columns
features = train_df.drop(columns=['id', 'mood', 'date'])
target = train_df['mood']  # Mood is now treated as a continuous variable

train_df = train_df.sort_values(by=["id", "date"])
train_df["next_day_mood"] = train_df.groupby("id")["mood"].shift(-1)
train_df = train_df.dropna(subset=["next_day_mood"])

features = train_df.drop(columns=["mood", "next_day_mood", "date", "id"])
target = train_df["next_day_mood"]


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

predictions = []
actuals = []
user_ids = []

# For each user
for user_id, group in train_df.groupby("id"):
    group = group.sort_values("date").copy()
    group["next_day_mood"] = group["mood"].shift(-1)
    group = group.dropna(subset=["next_day_mood"])

    if len(group) < 2:
       continue  # Not enough data for prediction

    X = group.drop(columns=["id", "date", "mood", "next_day_mood"])
    y = group["next_day_mood"]

    X_train = X.iloc[:-1]
    y_train = y.iloc[:-1]
    X_test = X.iloc[-1:]
    y_test = y.iloc[-1:]

    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    predictions.append(y_pred[0])
    actuals.append(y_test.values[0])
    user_ids.append(user_id)

# Evaluate across all users
print("MAE:", mean_absolute_error(actuals, predictions))
print("MSE:", mean_squared_error(actuals, predictions))
print("R²:", r2_score(actuals, predictions))

# Optional: view predictions
results_df = pd.DataFrame({
    "user_id": user_ids,
    "actual_next_day_mood": actuals,
    "predicted_next_day_mood": predictions
})
print(results_df)


MAE: 0.10376543209876422
MSE: 0.019428738683127184
R²: 0.930654113931799
    user_id  actual_next_day_mood  predicted_next_day_mood
0   AS14.01                  7.60                 7.543000
1   AS14.02                  7.20                 7.077500
2   AS14.03                  7.00                 7.156000
3   AS14.05                  7.20                 7.183500
4   AS14.06                  7.75                 7.741000
5   AS14.07                  6.60                 6.585000
6   AS14.08                  7.80                 7.569000
7   AS14.09                  6.60                 6.535000
8   AS14.12                  5.80                 6.067000
9   AS14.13                  7.20                 6.793500
10  AS14.14                  7.50                 7.441500
11  AS14.15                  7.00                 7.000000
12  AS14.16                  7.20                 6.944000
13  AS14.17                  7.20                 7.059000
14  AS14.19                  6.80         